In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

from utils.towncentre import load_towncentre, aug_data
from utils.angles import deg2bit, bit2deg, bit2deg_multi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True, verbose=1)

image_height, image_width = xtr.shape[1], xtr.shape[2]

ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)
yte_rad = np.deg2rad(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

Number of train samples: 6916
Number of validation samples: 874
Number of test samples: 904


#### We will model our distribution via

$ p(\phi | x) =  \sum_{i=1}^{k}{p(z_i|x) p(\phi|z_i,x)}$

In [28]:
from keras.layers.merge import concatenate
from keras import backend as K

In [46]:
from models.vgg_vmmix import BiternionVGGMixture
vgg_vmmix = BiternionVGGMixture()

TypeError: Output tensors to a Model must be Keras tensors. Found: Tensor("Reshape_45:0", shape=(?, 5, 1), dtype=float32)

In [19]:
xte_preds = vgg_vmmix.model.predict(xte)

In [35]:
a1 = K.reshape(vgg_vmmix.mu_preds_lst[0], [-1, 1, 2])
a2 = K.reshape(vgg_vmmix.mu_preds_lst[1], [-1, 1, 2])

In [37]:
concatenate([a1, a2], axis=1)

<tf.Tensor 'concatenate_15/concat:0' shape=(?, 2, 2) dtype=float32>

In [26]:
concatenate(vgg_vmmix.mu_preds_lst, axis=1)

IndexError: list assignment index out of range